In [2]:
import os
import numpy as np
import polars as pl

df = pl.read_csv('../Corpus/prod_dataset.csv')
df.head()

,query,prod_info,label
i64,str,str,f64
0,"""salon chair""","""[CLS] product_name: 21.7 '' w …",1.0
1,"""salon chair""","""[CLS] product_name: 22.5 '' wi…",0.0
2,"""salon chair""","""[CLS] product_name: 24.4 '' w …",1.0
3,"""salon chair""","""[CLS] product_name: 25 '' wide…",1.0
4,"""salon chair""","""[CLS] product_name: 27.6 '' w …",1.0


In [4]:
mapping = {
  1.0 : "Exact",
  0.1 : "Substitute",
  0.0 : "Irrelevant"
}

In [6]:
def create_documents(df: pl.DataFrame, mapping: dict[int, str], filepath: str = "../Corpus/corpus.txt") -> None:
  """
  Writes the content of dataframe to a .txt file in a directory
  Each row will be written as a separate line in the text file

  Params:
    df (pl.DataFrame): dataframe containing the content of a corpus
    dir (str): directory to write the text file
    filename (str): filename to write the text file

  Returns:
    None
  """
  with open(filepath, "w", encoding="utf-8") as file:
    for row in df.iter_rows(named=True):
      row = dict(row)
      if "label" in row and row["label"] in mapping:
        row["label"] = mapping[row["label"]]

      line = " | ".join(f"{col}: {val}" for col, val in row.items())
      file.write(line + "\n\n")

create_documents(df, mapping)

In [7]:
import logging

logging.info("Formed txt file from csv file")